# 3pt data preprocessing

In [1]:
import numpy as np
import pandas as pd
import sys
from copy import deepcopy

## Data import

In [2]:
sys.path.append(r"/Users/fordfishman/GitHub/3pt-shooting/code/python/")
# sys.path.append(r"/home/fordfishman/GitHub/3pt-shooting/code/python/")
from basketball_reference_scraper.teams import get_roster_stats



Set the seasons we desire and the teams as well. The seasons are 2010-2021, with each season going by the year at the time of the playoffs.

In [3]:
year_range = list( range(2010, 2022) ) 
team_abr = [
    'ATL',
    'BRK',
    'BOS',
    'CHA',
    'CHI',
    'CLE',
    'DAL',
    'DEN',
    'DET',
    'GSW',
    'HOU',
    'IND',
    'LAC',
    'LAL',
    'MEM',
    'MIA',
    'MIL',
    'MIN',
    'NOP',
    'NYK',
    'OKC',
    'ORL',
    'PHI',
    'PHO',
    'POR',
    'SAC',
    'SAS',
    'TOR',
    'UTA',
    'WAS',
    ]




In [4]:
def player_index(names, seasons):

    indices = list()

    for i in range(len(names)):

        name = names[i].replace(' ', '')
        indices.append( '%s%s' % (name, seasons[i]) )

    return indices
    
def get_data(mode):

    df = get_roster_stats('GSW', 2021, mode)
    df = pd.DataFrame().reindex(columns=df.columns)

    for year in year_range:
        for team in team_abr:

            df_i = get_roster_stats(team, year, mode)
            df_i['index'] = player_index(df_i.PLAYER, df_i.SEASON)
            df = df.append(df_i, ignore_index=True)

    return df


Run API and save raw data as .csv files. Only run if necessary, as it takes over an hour.

In [5]:
df_adv = get_data('ADVANCED')
df_tot = get_data('TOTALS')
df_poss = get_data('PER_POSS')
df_adv.to_csv("~/GitHub/3pt-shooting/data/allplayer_adv.csv", index=False)
df_poss.to_csv("~/GitHub/3pt-shooting/data/allplayer_poss.csv", index=False)
df_tot.to_csv("~/GitHub/3pt-shooting/data/allplayer_tot.csv", index=False)

Read in data. Using the raw API functions have issues with converting to the correct datatypes.

In [56]:
df_poss = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_poss.csv")
df_adv = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_adv.csv")
df_tot = pd.read_csv("~/GitHub/3pt-shooting/data/allplayer_tot.csv")


Remove columns with no data.

In [57]:
df_poss = df_poss.drop(['Unnamed: 29'], axis=1)
df_adv = df_adv.drop(['Unnamed: 19', 'Unnamed: 24'], axis=1)

remove duplicate columns, print out duplicates

In [58]:
for col in df_tot:
    
    if col in df_poss and df_tot[col].equals(df_poss[col]):

        df_poss = df_poss.drop([col], axis=1)
        
    if col in df_adv and df_tot[col].equals(df_adv[col]):
        
        df_adv = df_adv.drop([col], axis=1)


rename possession columns

In [59]:
df_poss = df_poss.add_suffix('_poss')


combine data frames

In [60]:
df_totposs = df_tot.join(df_poss, how='outer')
df_all = df_totposs.join(df_adv, how='outer')
df_all.columns


Index(['PLAYER', 'POS', 'AGE', 'TEAM', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'SEASON',
       'index', 'FG_poss', 'FGA_poss', '3P_poss', '3PA_poss', '2P_poss',
       '2PA_poss', 'FT_poss', 'FTA_poss', 'ORB_poss', 'DRB_poss', 'TRB_poss',
       'AST_poss', 'STL_poss', 'BLK_poss', 'TOV_poss', 'PF_poss', 'PTS_poss',
       'ORtg_poss', 'DRtg_poss', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%',
       'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS', 'WS',
       'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [61]:
df_trade = deepcopy(df_all)

idents = dict() # dictionary of id to list of indexes

for i, row in df_all.iterrows():

    ident = row['index']

    if not ident in idents:
        idents[ident] = [i]

    else:
        idents[ident].append(i)




need to fix so that each player year is a single row

In [62]:
num = df_trade.select_dtypes(include=['float64','int64']).columns
char = df_trade.select_dtypes(include=['object']).columns
totals = pd.Index(['G', 'GS', 'MP', 'FG', 'FGA', 
       '3P', '3PA', '2P', '2PA',  'FT', 'FTA', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'])
aves_list = list()

for i in num:
    if i not in totals:
        aves_list.append(i)
aves = pd.Index(aves_list)

all_inds = list()

i = 0
for ident, inds in idents.items():

    inds = list(inds)

    if len(inds) > 1: # if a player was moved
        all_inds += inds
        
        char_i = df_trade.loc[inds, char].reset_index(drop=True)
        tot_i = df_trade.loc[inds, totals].reset_index(drop=True)
        ave_i = df_trade.loc[inds, aves].reset_index(drop=True)

        char_new = char_i.loc[0,:]
        tot_new = tot_i.sum(axis=0)
        # print(char_i)
        # break
        # print(tot_new)

        games = tot_new.G
        games_r = tot_i.G/games # ratio of games played
        ave_new = np.matmul(games_r, ave_i)

        new_row = pd.concat([char_new, tot_new, ave_new], axis=0)
        # print(new_row)
        
        df_trade = df_trade.append(new_row, ignore_index = True)

df_trade = df_trade.drop(index=all_inds).reset_index(drop=True)



# print(m)

/var/folders/qb/f7nlg06s14v9pk0mfmmc714m0000gn/T/ipykernel_16530/118433818.py:35: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  ave_new = np.matmul(games_r, ave_i)


remove seasons below 100 3PTA

In [63]:
df_limit = df_trade.loc[df_trade['3PA']>=100].reset_index(drop=True)

Add information about next year's shooting

In [64]:

seasons_sorted = sorted(list(set(df_limit.SEASON)))
player_dict = dict()

next_3pt = list()
next_imp = list()

for i in range(len(df_limit)):

    player = df_limit.loc[i, "PLAYER"]
    season = df_limit.loc[i, "SEASON"]
    pct = df_limit.loc[i, "3P%"]

    if player not in player_dict:
        player_dict[player] = {sea: np.nan for sea in seasons_sorted}
        player_dict[player][season] = pct
    
    else:
        player_dict[player][season] = pct

seasons_to_keep = dict() # a dictionary of season IDs we will keep

for player, seasons in player_dict.items():

    for i, season in enumerate(seasons_sorted): # iterate over all seasons in order

        if i == len(seasons_sorted) - 1: #if its the last season
            break

        season_1 = seasons_sorted[i+1]

        if not np.isnan(seasons[season]) and not np.isnan(seasons[season_1]):  

           name = player.replace(' ', '')
           index = '%s%s' % (name, season)
           seasons_to_keep[index] = seasons[season_1]

            


Add in next season values. This, along with the differential will be the targets.

In [65]:
df_limit['next_3P%'] = np.nan

for ident, pct in seasons_to_keep.items():
    i = df_limit[df_limit['index']==ident].index
    df_limit.loc[i, 'next_3P%'] = pct

df_final = df_limit.dropna(subset=['next_3P%'])
df_final['3pt_dif'] = df_final['next_3P%'] - df_final['3P%'] # differential between this and last seasons shooting percentage

/var/folders/qb/f7nlg06s14v9pk0mfmmc714m0000gn/T/ipykernel_16530/1234598018.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['3pt_dif'] = df_final['next_3P%'] - df_final['3P%'] # differential between this and last seasons shooting percentage


change season to be a numerical

In [66]:
def season_num(season):
    year = season[-2:]
    return int( '20%s' % year )

season_v = map(season_num, df_final.SEASON)
df_final['SEASON'] = list(season_v)

/var/folders/qb/f7nlg06s14v9pk0mfmmc714m0000gn/T/ipykernel_16530/3157018658.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['SEASON'] = list(season_v)


Save final data frame

In [68]:
df_final.to_csv("~/GitHub/3pt-shooting/data/preprocessed.csv", index=False)

In [69]:

sys.path.append(r"/Users/fordfishman/GitHub/3pt-shooting/code/python/")
# sys.path.append(r"/home/fordfishman/GitHub/3pt-shooting/code/python/")
from basketball_reference_scraper.teams import get_roster_stats

get_roster_stats('SAS', 2017, 'PER_GAME')
# h = HTMLParser()
# print(h.unescape('&pound;682m'))
# x = h.unescape('Tom&Atilde;&iexcl;&Aring;&iexcl; Satoransk&Atilde;&frac12;')
# print(df)
# remove_accents2(df.PLAYER, 'SAS', 2017)

# "https://widgets.sports-reference.com/wg.fcgi?css=1&site=bbr&url=%2Fleagues%2FNBA_2017_per_game.html&div=div_per_game_stats"

,PLAYER,POS,AGE,TEAM,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,SEASON
0,LaMarcus Aldridge,PF,31,SAS,72,72,32.4,6.9,14.6,.477,...,2.4,4.9,7.3,1.9,0.6,1.2,1.4,2.2,17.3,2016-17
1,Kyle Anderson,SF,23,SAS,72,14,14.2,1.3,2.9,.445,...,0.5,2.4,2.9,1.3,0.7,0.4,0.5,0.9,3.4,2016-17
2,Joel Anthony,C,34,SAS,19,0,6.4,0.5,0.8,.625,...,0.4,1.2,1.6,0.2,0.1,0.3,0.2,0.6,1.3,2016-17
3,Dāvis Bertāns,PF,24,SAS,67,6,12.1,1.5,3.5,.440,...,0.3,1.1,1.5,0.7,0.3,0.4,0.5,1.1,4.5,2016-17
4,Dewayne Dedmon,C,27,SAS,76,37,17.5,2.1,3.4,.622,...,1.7,4.8,6.5,0.6,0.5,0.8,0.8,2.4,5.1,2016-17
5,Bryn Forbes,SG,23,SAS,36,0,7.9,1.0,2.8,.364,...,0.1,0.6,0.6,0.6,0.0,0.0,0.3,0.6,2.6,2016-17
6,Pau Gasol,C,36,SAS,64,39,25.4,4.7,9.4,.502,...,1.7,6.2,7.8,2.3,0.4,1.1,1.3,1.7,12.4,2016-17
7,Manu Ginóbili,SG,39,SAS,69,0,18.7,2.5,6.4,.390,...,0.4,1.9,2.3,2.7,1.2,0.2,1.4,1.7,7.5,2016-17
8,Danny Green,SG,29,SAS,68,68,26.6,2.6,6.6,.392,...,0.5,2.8,3.3,1.8,1.0,0.8,1.1,1.8,7.3,2016-17
9,Nicolás Laprovíttola,PG,27,SAS,18,3,9.7,1.1,2.6,.426,...,0.1,0.5,0.6,1.6,0.2,0.1,1.1,1.1,3.3,2016-17
